# Análise Preditiva de Suicídios

## Descrição do dataset
Os dados foram coletados em forma de questionário através de redes sociais e emails. Alguns direcionados a grupos específicos de apoio (contra depressão). As questões abordadas foram baseadas nesta [pesquisa](https://www.kaggle.com/kingburrito666/the-demographic-rforeveralone-dataset). <br>

<b>Descrição das variáveis:</b><br>
* Q1: "Qual a sua idade?" <br>
* Q2: "Qual gênero você mais se identifica?" <br>
* Q3: "Qual sua orientação  sexual?" <br>
* Q4: "Qual o seu nível de renda anual?" <br>
* Q5: "Qual sua etnicidade?" <br>
* Q6: "Como você descreve sua massa corporal?" <br>
* Q7: "Você é virgem?" <br>
* Q8: "Você pagaria para ter uma relação sexual?" <br>
* Q9: "Quantos amigos você tem na vida real?" <br>
* Q10: "Você possui ansiedade/fobia social?" <br>
* Q11: "Você possui algum tipo de diagnóstico de transtorno mental?" <br>
* Q12: "Que tipo de ajuda você quer?" <br>
* Q13: "Você já tentou se suicidar?" <br>
* Q14: "Qual seu status empregatício? Atualmente você é..." <br>
* Q15: "Qual seu cargo atual?" <br>
* Q16: "Qual seu nível de educação?" <br>
* Q17: "O que você fez para se sentir melhor consigo mesmo?" <br>
* Q18: "Com quem você mora?" <br>
* Q19: "Seus pais são divorciados?" <br>


In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 100)

## Funções úteis

In [ ]:
from plot_metric.functions import BinaryClassification

In [ ]:
def plot_roc_curve(ytest, predicted_yproba, figure_name):
    %matplotlib inline
    bc = BinaryClassification(ytest, predicted_yproba, labels=["Classe 0", "Classe 1"])
    plt.figure(figsize=(5,5))
    bc.plot_roc_curve()
    plt.savefig(figure_name)
    plt.show()
    
    
def plot_resampling_comparing(confusion_matriz_os, confusion_matriz_us, figure_name):
    fig = plt.figure(figsize=(10,4))

    ax0 = fig.add_subplot(121)
    ax0.set_title('Oversampling')
    ax1 = fig.add_subplot(122)
    ax1.set_title('Undersampling')

    sns.heatmap(confusion_matriz_os, annot=True, cmap=sns.light_palette("black"), ax=ax0)
    sns.heatmap(confusion_matriz_us, annot=True, cmap=sns.light_palette("black"), ax=ax1)
    
    plt.savefig(figure_name)

In [ ]:
def check_classifiers_scores(classifiers, Xtrain, ytrain):
    for key, classifier in classifiers.items():
        classifier.fit(Xtrain, ytrain)
        training_score = cross_val_score(classifier, Xtrain, ytrain, cv=5, n_jobs=-1)
        precision_score = cross_val_score(classifier, Xtrain, ytrain, cv=5, n_jobs=-1, scoring='precision')
        recall_score = cross_val_score(classifier, Xtrain, ytrain, cv=5, n_jobs=-1, scoring='recall')

        print(classifier.__class__.__name__, 
              "[Accuracy]:", round(training_score.mean(), 2) * 100, "%", 
              "[Precision]:", round(precision_score.mean(), 2),
              "[Recall]:", round(recall_score.mean(), 2)
        )

In [ ]:
def show_feature_importance(importances, current_df):
    indices = np.argsort(importances)[::-1]
    print("Feature ranking:")

    for f in range(current_df.shape[1]):
        print("%d. %s (%f)" % (f + 1, current_df.columns[indices[f]], importances[indices[f]]))


def plot_feature_importance(importances, current_df, figure_name):
    indices = np.argsort(importances)
    f, ax = plt.subplots(figsize=(15, 8))
    ax = plt.gca()
    ax.set_facecolor('white')

    plt.title("Feature ranking", fontsize = 20)
    plt.barh(range(current_df.shape[1]), importances[indices], color="black", align="center")

    plt.yticks(range(current_df.shape[1]), current_df.columns[indices])
    plt.ylim([-1, current_df.shape[1]])
    plt.ylabel("Features", fontsize = 20)
    plt.xlabel("Importance", fontsize = 20)
    plt.savefig(figure_name)

## Dataset

In [ ]:
survey_data = pd.read_csv('survey_data.csv')

In [ ]:
df = survey_data.iloc[:, 1:20]
df.columns = ['Q{}'.format(x+1) for x in range(0, 19)]
df.isnull().sum()

## 1. Pré-processamento

In [ ]:
def check_values_column(series):
    return pd.DataFrame(series.str.strip().unique()).sort_values(by=0)

In [ ]:
def replace_values(df, column_name, values_search, value_replace):
    for value in values_search:
        contains_value = df[column_name].str.lower().str.contains(value)
        df[column_name] = np.where(contains_value == True, value_replace, df[column_name])

In [ ]:
def create_age_range(df, feature_name, age_start, age_end):
    current = age_start
    step = 4
    
    while (current < age_end):
        age_endd = current + step
        
        df.loc[df.Q1.between(current, age_endd), feature_name] = \
        '{} à {} anos'.format(current, age_endd)

        current += (1 + step)


### 1.1 Variável Q1 - "Qual a sua idade?"
Limpeza e feature para a variável idade. <br>

In [ ]:
fig = plt.figure(figsize=(10,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122)

sns.distplot(df.Q1, kde=True, color='Black', ax=ax0);
sns.boxplot(x=df["Q1"], palette=sns.light_palette("black"), orient='v', ax=ax1)

plt.savefig('q1v2.png')

fig.show()

In [ ]:
df.Q1.describe()

#### 1.1.1 Feature por intervalo de idade

In [ ]:
#create_age_range(df, 'Q1_FAIXA_IDADE', 10, 49)

In [ ]:
#df.Q1_FAIXA_IDADE.value_counts()

### 1.2 Variável Q2 - "Qual gênero você mais se identifica?"

In [ ]:
df.Q2.value_counts()

In [ ]:
replace_values(df, 'Q2', ['agenero', 'não binário'], 'Não binário')

In [ ]:
sns.countplot(x='Q2', data=df, palette='Blues')
plt.savefig('q2.png')

### 1.3 Variável Q3 - "Orientação sexual"
Limpeza e recategorização

In [ ]:
df.Q3.value_counts(), df[df.Q3.isna()].shape

In [ ]:
replace_values(df, 'Q3', ['pansexual', 'panssexual'], 'Pansexual')
replace_values(df, 'Q3', ['bi-curioso', 'não sei'], 'Outros')

In [ ]:
chart_q3 = sns.countplot(x='Q3', data=df, palette='Blues', order=df.Q3.value_counts().index)
chart_q3.set_xticklabels(chart_q3.get_xticklabels(), rotation=20)

plt.savefig('q3.png')

### 1.4 Variável Q4 - "Qual o seu nível de renda anual?"

In [ ]:
df[df['Q4'].isna()]['Q14']

In [ ]:
df['Q4'].value_counts()

In [ ]:
chart_q4 = sns.countplot(x='Q4', data=df, palette='Blues', order=df.Q4.value_counts().index)
chart_q4.set_xticklabels(chart_q4.get_xticklabels(), rotation=90)
plt.savefig('q4.png')

#### 1.4.1 Substituindo valores pela moda

Baseado na análise, os registros são referente a estudantes e/ou desempregados

In [ ]:
mode = df['Q4'].value_counts().index[0]

In [ ]:
df['Q4'].fillna(mode, inplace=True)

### 1.5 Variável Q5 - "Qual sua etnicidade?"

In [ ]:
chart_q5 = sns.countplot(x='Q5', data=df, palette='Blues', order=df.Q5.value_counts().index)
chart_q5.set_xticklabels(chart_q5.get_xticklabels(), rotation=15)
plt.savefig('q5.png')

### 1.6 Variável Q6 - "Como você descreve sua massa corporal?"

In [ ]:
df.Q6.value_counts()

In [ ]:
chart_q6 = sns.countplot(x='Q6', data=df, palette='Blues', order=df.Q6.value_counts().index)
chart_q6.set_xticklabels(chart_q6.get_xticklabels(), rotation=15)
plt.savefig('q6.png')

### 1.7 Variável Q7 - "Você é virgem?"

In [ ]:
sns.countplot(x='Q7', data=df, palette='Blues', order=df.Q7.value_counts().index)
plt.savefig('q7.png')

In [ ]:
df.Q7 = np.where(df.Q7 == 'Sim', 1, 0)

### 1.8 Variável Q8 - "Você pagaria para ter uma relação sexual?"

In [ ]:
df.Q8.value_counts()

In [ ]:
replace_values(df, 'Q8', ['cogitei'], 'Talvez')
replace_values(df, 'Q8', ['ganhasse'], 'Sim')

In [ ]:
sns.countplot(x='Q8', data=df, palette='Blues', order=df.Q8.value_counts().index)
plt.savefig('q8.png')

### 1.9 Variável Q9 - "Quantos amigos você tem na vida real?"

In [ ]:
df.Q9.describe()

In [ ]:
ax = sns.distplot(df.Q9, bins=20, kde=False, color='Black');
ax.set_yscale('log')

In [ ]:
df[df['Q9'] <= 20].Q9.describe()

#### 1.9.1 Atribuindo a média sem outliers
Para os registros com valores maiores que 20 amigos, é substituído o número de amigos
pela média considerando até 20 amigos.

In [ ]:
friends_mean_without_outliers = df[df['Q9'] <= 20].Q9.mean()

In [ ]:
df['Q9_PROCESSADO'] = np.where(df['Q9'] > 20, friends_mean_without_outliers, df['Q9'])

In [ ]:
df['Q9_PROCESSADO'].describe()

In [ ]:
ax = sns.distplot(df.Q9_PROCESSADO, bins=20, kde=False, color='Black');
ax.set_yscale('log')

In [ ]:
fig = plt.figure(figsize=(10,4))

ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122)
ax0.set_yscale('log')
ax1.set_yscale('log')

sns.distplot(df.Q9, kde=False, bins=20, color='Black', ax=ax0);
sns.distplot(df.Q9_PROCESSADO, bins=20, kde=False, color='Black', ax=ax1);

plt.savefig('q9.png')

fig.show()

In [ ]:
df = df.drop(columns='Q9')

### 1.10 Variável Q10 - "Você possui ansiedade/fobia social?"

In [ ]:
sns.countplot(x='Q10', data=df, palette='Blues', order=df.Q10.value_counts().index)
plt.savefig('q10.png')

In [ ]:
df.Q10 = np.where(df.Q10 == 'Sim', 1, 0)

### 1.11 Variável Q11 - "Você possui algum tipo de diagnóstico de transtorno mental?"
Esta variável envolve multipla seleção e digitação em campo livre.<br>
Os passos para tratamento serão:<br>
* Atribuir valor caso não tenha sido informado;
* Sobrescrever valores redundantes (digitados);
* Converter multipla seleção em colunas separadas;
* Converter as classes separadas para binário;
* Classificar valores digitados manualmente em categorias existentes e/ou criando novas (se necessário);

In [ ]:
df[df['Q11'].isna()].shape

In [ ]:
df['Q11'].fillna('Sem resposta', inplace=True)

In [ ]:
check_values_column(df['Q11'])

#### 1.11.1 Agrupando/sobrescrevendo valores redundantes

In [ ]:
nenhum = 'Nenhum'

df.loc[df.Q11.str.lower().str.contains('nenhum'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('não'), 'Q11'] = nenhum
df.loc[df.Q11.str.lower().str.contains('sem diagnóstico médico'), 'Q11'] = nenhum

outros = 'Outros'
df.loc[df.Q11.str.lower().str.contains('ansiedade de leve'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('apenas períodos de ansiedade'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('problemas psicológicos após lesão corporal'), 'Q11'] = outros
df.loc[df.Q11.str.lower().str.contains('tdah'), 'Q11'] = outros

df.Q11 = df.Q11.str.strip().str.replace('toc', 'TOC', regex=True)
df.Q11 = df.Q11.str.strip().str.replace('TOC', 'Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)', regex=True)

In [ ]:
check_values_column(df['Q11'])

#### 1.11.2 Transformando variáveis categóricas em binário
Criando colunas binárias separadas para cada tipo de transtorno.

In [ ]:
df_Q11 = pd.get_dummies(df['Q11'].str.strip().str.split(';', expand=True))

df['Q11_T1'] = df_Q11[['0_Transtornos Alimentares  (Anorexia, Bulimia, Transtorno de Ruminação, etc...)']].max(axis=1)

df['Q11_T2'] = df_Q11[['0_Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)',
                       '1_Transtornos de Ansiedade (Fobia Específica, Transtorno de Ansiedade Generalizada, Transtorno de Pânico, etc...)']].max(axis=1)

df['Q11_T3'] = df_Q11[['0_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)',
                       '1_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)',
                       '2_Transtorno Bipolar e Transtornos Relacionados (Transtorno Bipolar Tipo I, Transtorno Bipolar Tipo II, Transtorno Ciclotímico, etc...)']].max(axis=1)

df['Q11_T4'] = df_Q11[['0_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '1_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '2_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)',
                       '3_Transtornos Depressivos (Transtorno Disruptivo da Desregulação do Humor, Transtorno Depressivo Persistente (Distimia), etc...)']].max(axis=1)

df['Q11_T5'] = df_Q11[['0_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '1_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '2_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '3_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)',
                       '4_Transtornos da Personalidade (Antissocial, Borderline, Esquizoide, Narcisista, Paranoide, etc...)']].max(axis=1)

df['Q11_T6'] = df_Q11[['0_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '1_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '2_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '3_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '4_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)',
                       '5_Transtornos Relacionados a Trauma e a Estressores (Transtorno de Apego Reativo, Transtorno de Interação Social Desinibida, Transtorno de Estresse Pós-traumático, Transtorno de Estresse Agudo, etc..)']].max(axis=1)


In [ ]:
df['Q11_OUTROS'] = df_Q11[['0_Outros']].max(axis=1)
df['Q11_SEM_RESPOSTA'] = df_Q11[['0_Sem resposta']].max(axis=1)

In [ ]:
df = df.drop(columns='Q11')

In [ ]:
df.columns

In [ ]:
df[df.Q11_OUTROS == 1]

### 1.12 Variável Q12 - "Que tipo de ajuda você quer?"

In [ ]:
df.Q12.value_counts()

In [ ]:
df_Q12 = pd.get_dummies(df['Q12'].str.strip().str.split(';', expand=True))
df_Q12

In [ ]:
df['Q12_N1'] = df_Q12[['0_Amigos', '3_Amigos compreensivos e, acima, de tudo, uma sociedade mais justa e compreensiva']].max(axis=1)
df['Q12_N2'] = df_Q12[['0_Coaching de encontro', '1_Coaching de encontro']].max(axis=1)
df['Q12_N3'] = df_Q12[['0_Conselheiro', '1_Conselheiro', '2_Conselheiro']].max(axis=1)
df['Q12_N4'] = df_Q12[['0_Encontro arranjado', '1_Encontro arranjado', '2_Encontro arranjado']].max(axis=1)
df['Q12_N5'] = df_Q12[['0_Não quero ajuda', '1_Não quero ajuda']].max(axis=1)
df['Q12_N6'] = df_Q12[['0_Não sei', '1_Não sei', '2_Não sei', '3_Não sei']].max(axis=1)
df['Q12_N7'] = df_Q12[['0_Terapia', '1_Terapia', '2_Terapia', '3_Terapia', '4_Terapia', 
                       '1_Já faço terapia e acompanhamento psiquiátrico', 
                       '1_eu já faço terapia. Obg', '3_Já faço medicamentos e já faço terapia']].max(axis=1)

In [ ]:
df['Q12_MEDICACAO'] = df_Q12[['2_Medicação', '3_Já faço medicamentos e já faço terapia']].max(axis=1)
df['Q12_PSIQUIATRA'] = df_Q12[['1_Já faço terapia e acompanhamento psiquiátrico', 
                               '2_Psiquiatra, neurologista', '2_Psiquiatria']].max(axis=1)
df['Q12_OUTROS'] = df_Q12['4_Mais curiosidade sobre o que são do que necessitar ajuda']

In [ ]:
df = df.drop(columns='Q12')

In [ ]:
df.head()

### 1.13 Variável Q13 - "Você já tentou se suicidar?"
Label/target da pesquisa

In [ ]:
chart_q13 = sns.countplot(x='Q13', data=df, palette='Blues', order=df.Q13.value_counts().index)
plt.savefig('q13.png')

In [ ]:
df.Q13.value_counts() / df.Q13.count()

In [ ]:
df.Q13 = np.where(df.Q13 == 'Sim', 1, 0)

### 1.14 Variável Q14 - "Qual seu status empregatício? Atualmente você é...?"

In [ ]:
chart_q14 = sns.countplot(x='Q14', data=df, palette='Blues', order=df.Q14.value_counts().index)
chart_q14.set_xticklabels(chart_q14.get_xticklabels(), rotation=90)
plt.savefig('q14.png')

### 1.15 Variável Q15 - "Qual seu cargo atual?"

In [ ]:
df['Q15'].str.strip().value_counts()

#### 1.15.1 Agrupando valores

In [ ]:
desenvolvimento = ['desenvolvedor', 'dev', 'analista', 'software', 'inteligência artificial']
ciencia_de_dados = ['dados', 'estatístico']
estudante = ['estudante', 'caloura', 'bolsista', 'escola', 'terceiro ano']
professor = ['professor', 'ministro aula casualmente']
estagiario = ['estagiári', 'estágio']
vendas = ['vendedor', 'vendas', 'atendente', 'call center', 'marketing', 'operador de caixa']
recepcionista = ['recepcionista', 'recepçao']
autonomo = ['dono', 'autônoma', 'empreendedor', 'empresária']
lideranca = ['gerente', 'diretor', 'líder']

replace_values(df, 'Q15', desenvolvimento, 'Desenvolvimento de Software')
replace_values(df, 'Q15', estudante, 'Estudante')
replace_values(df, 'Q15', estagiario, 'Estágio')
replace_values(df, 'Q15', professor, 'Educação')
replace_values(df, 'Q15', vendas, 'Vendas / Telemarketing')
replace_values(df, 'Q15', recepcionista, 'Recepcionista')
replace_values(df, 'Q15', autonomo, 'Empresário(a) / Empreendedor(a) / Autônomo(a)')
replace_values(df, 'Q15', ciencia_de_dados, 'Ciência de Dados')
replace_values(df, 'Q15', lideranca, 'Diretoria / Gerência / Liderança')
replace_values(df, 'Q15', ['redator'], 'Redator(a)')
replace_values(df, 'Q15', ['psic'], 'Psicologia')
replace_values(df, 'Q15', ['pesquisador'], 'Pesquisador(a)')
replace_values(df, 'Q15', ['coordenad'], 'Coordenador(a)')
replace_values(df, 'Q15', ['adm'], 'Administrativo')
replace_values(df, 'Q15', ['arquitet'], 'Arquiteto(a)')

df['Q15'].fillna('Sem resposta', inplace=True)

In [ ]:
df.describe()

In [ ]:
df[df['Q15'] == 'Sem resposta'].shape

In [ ]:
max(df.isnull().sum())

In [ ]:
df.head()

### 1.16 Variável Q16 - "Qual seu nível de educação?"

In [ ]:
df.Q16.value_counts()

In [ ]:
replace_values(df, 'Q16', ['estou cursando o ensino medio', 'cursando o ensino médio.'], 'Ensino médio incompleto')
replace_values(df, 'Q16', ['superior quase completo'], 'Superior (graduação) incompleto')

### 1.17 Variável Q17 - "O que você fez para se sentir melhor consigo mesmo?

In [ ]:
df.Q17.value_counts()

In [ ]:
df_Q17 = pd.get_dummies(df['Q17'].str.strip().str.split(';', expand=True))
df_Q17.columns

In [ ]:
df['Q17_A1'] = df_Q17['0_Dieta']
df['Q17_A2'] = df_Q17[['0_Indo à academia', '1_Indo à academia']].max(axis=1)
df['Q17_A3'] = df_Q17[['0_Nada', '1_Nada', '2_Nada']].max(axis=1)
df['Q17_A4'] = df_Q17[['0_Outros exercícios físicos', '1_Outros exercícios físicos', 
                       '2_Outros exercícios físicos', '3_Outros exercícios físicos',
                       '1_Corrida, jogos eletrônicos, leitura, culinária', 
                       '1_Exercício físico coletivo. Futebol, vôlei,etc']].max(axis=1)

df['Q17_A5'] = df_Q17[['0_Participando de encontros/clubes sociais', 
                       '1_Participando de encontros/clubes sociais',
                       '2_Participando de encontros/clubes sociais',
                       '3_Participando de encontros/clubes sociais',
                       '4_Participando de encontros/clubes sociais']].max(axis=1)

df['Q17_A6'] = df_Q17[['0_Procedimento cosmético', 
                       '1_Procedimento cosmético',
                       '2_Procedimento cosmético',
                       '3_Procedimento cosmético']].max(axis=1)

df['Q17_A7'] = df_Q17[['0_Terapia', '1_Terapia', '2_Terapia', '3_Terapia', '4_Terapia', '5_Terapia',
                       '2_Conversar com amigos, jogar, terapia ocupacional, pintar, artesanato, escrever, comprar roupas que me fazem confortável']].max(axis=1)

df['Q17_A8'] = df_Q17[['0_Voluntariado', '1_Voluntariado', '2_Voluntariado', '3_Voluntariado', 
                       '4_Voluntariado', '5_Voluntariado', '6_Voluntariado']].max(axis=1)

In [ ]:
df['Q17_MUSICA_TEATRO_ARTE'] = df_Q17[['0_Canto e ouço músicas', '1_Tocar instrumentos/recorrer aos amigos',
                                       '0_Fumo maconha, toco, fumo maconha, transo', 
                                       '0_Ouvir musicas animadas, assistir series que eu amo',
                                       '0_Pintar e Dormir', '1_Grupo de teatro', '1_Teatro',
                                       '1_fazendo coisas sozinha, ouvindo música, desenhando...',
                                       '2_Conversar com amigos, jogar, terapia ocupacional, pintar, artesanato, escrever, comprar roupas que me fazem confortável', 
                                       '3_Dança, Piano', '3_Escrita']].max(axis=1)

df['Q17_AMIGOS'] = df_Q17[['0_Falo com meus amigos, me distraio com qualquer coisa', 
                           '1_Tocar instrumentos/recorrer aos amigos', '0_Saindo com amigos',
                           '2_Conversar com amigos, jogar, terapia ocupacional, pintar, artesanato, escrever, comprar roupas que me fazem confortável'
                          ]].max(axis=1)

df['Q17_DROGAS'] = df_Q17[['0_Fumo maconha, toco, fumo maconha, transo', '0_maconha', 
                           '3_Tracei metas e acatei práticas que me agradam, independente do consenso social sobre estas (fumar)']].max(axis=1)

df['Q17_JOGOS'] = df_Q17[['0_Jogos', '1_Corrida, jogos eletrônicos, leitura, culinária', '1_Jogos', 
                          '2_Conversar com amigos, jogar, terapia ocupacional, pintar, artesanato, escrever, comprar roupas que me fazem confortável']].max(axis=1)

df['Q17_LEITURA'] = df_Q17[['0_Leitura e passes no centro espírita', 
                            '1_Corrida, jogos eletrônicos, leitura, culinária', 
                            '2_Ler, obter conhecimento']].max(axis=1)

df['Q17_OUTROS'] = df_Q17[['0_Dormir', '0_Meu filho e ficando de boas',
                           '0_Pilates, yoga, meditação.', '0_Pintar e Dormir',
                           '0_Procedimentos estéticos, ajudo minha amiga na clínica dela todas as manhãs',
                           '0_Qualquer coisa que me tire da realidade', '0_Resolver os problemas pessoais',
                           '0_Ver pontos positivos rotineiramente', '0_compras',
                           '0_foquei em fazer coisas que gostaria de fazer há tempos mas não fazia por achar que era incapaz ou que não estava pronta para fazê-las.',
                           '1_Corrida, jogos eletrônicos, leitura, culinária',
                           '1_Eu me aceito como sou,ainda mais porque tenho pessoas maravilhosas comigo então não tem como eu querer fazer algo para me aceitar além do mais Deus me fez assim do jeitinho que sou então sou grata por tudo que tenho e sou.',
                           '1_Me afastando da minha mãe socialmente paranóica e pressionadora',
                           '1_Tento relaxar sozinha, pensar na raiz do problema e tentar melhorar',
                           '1_Trabalhar', '1_fazendo coisas sozinha, ouvindo música, desenhando...',
                           '2_Assistindo porno', '2_Conhecendo pessoas que eu tenha uma conexão e me entendam',
                           '2_Conversar com amigos, jogar, terapia ocupacional, pintar, artesanato, escrever, comprar roupas que me fazem confortável', 
                           '2_Tenho hobbies', '3_Atendimento psiquiátrico', '3_Meus gatos',
                           '0_Leitura e passes no centro espírita'
                          ]].max(axis=1)


In [ ]:
df = df.drop(columns='Q17')

### 1.18 Variável Q18 - "Com quem você mora?"

In [ ]:
chart_q18 = sns.countplot(x='Q18', data=df, palette='Blues', order=df.Q18.value_counts().index)
chart_q18.set_xticklabels(chart_q18.get_xticklabels(), rotation=90)
plt.savefig('q18.png')

### 1.19 Variável Q19 - "Seus pais são divorciados?"

In [ ]:
sns.countplot(x='Q19', data=df, palette='Blues', order=df.Q19.value_counts().index)

In [ ]:
df.Q19.value_counts() / df.Q19.count()

In [ ]:
df.Q19 = np.where(df.Q19 == 'Sim', 1, 0)

In [ ]:
df.head()

In [ ]:
df.shape

## 2. Análise Exploratória

In [ ]:
df.describe()

### 2.1 Convertendo Variáveis Categóricas

In [ ]:
df = pd.get_dummies(df)

In [ ]:
df.iloc[131]

In [ ]:
df.shape

### 2.2 Correlações

In [ ]:
#Renomeando para melhor visualização

df['Q11_SR'] = df.pop('Q11_SEM_RESPOSTA')

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(50,50))

corr = df.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)
ax1.set_title("Matriz de correlações \n", fontsize=14)

In [ ]:
corr_target = abs(corr["Q13"])

#Selecionado somente possíveis correlações com a variável label
relevant_features = corr_target[corr_target > 0.2]

In [ ]:
df[relevant_features.index].corr()

In [ ]:
df.head()

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(12, 8))

relevant_corr = df[relevant_features.index].corr()
chart = sns.heatmap(relevant_corr, cmap='coolwarm_r', annot_kws={'size':20}, ax=ax1)

ax1.set_title("Matriz de correlações \n", fontsize=14)

chart.set_xticklabels(chart.get_xticklabels(), rotation=45)

plt.savefig('correlation_greater_02.png')

In [ ]:
df.shape

## 3. Seleção de variáveis

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
backup_df = df.copy()

In [ ]:
y = df.pop('Q13')

### 3.1 Greedy Feature Elimination
(Similar to Backward Stepwise Selection)

In [ ]:
#estimator = SVR(kernel="linear")
#selector = RFE(estimator, 20, step=1)
#selector = selector.fit(X, y)

In [ ]:
#selector.support_

In [ ]:
#selector.ranking_

In [ ]:
#selector.get_params

In [ ]:
#support = pd.Series(selector.support_)

In [ ]:
#df.columns[support.index[support]]

In [ ]:
#df[df.columns[support.index[support]]]

### 3.2 Variance Threshold

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(df)

In [ ]:
sel.variances_

In [ ]:
sel.get_support()

In [ ]:
support = pd.Series(sel.get_support())

In [ ]:
featured_df = df[df.columns[support.index[support]]]

In [ ]:
featured_df

In [ ]:
featured_df.columns

In [ ]:
X = featured_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [ ]:
print("Full Dataset:", X.shape)
print("Treinamento:", X_train.shape)
print("Test:", X_test.shape)

In [ ]:
backup_df['Q13'].value_counts() / backup_df.shape[0]

In [ ]:
y_train.value_counts() / y_train.shape

## 4. Desenvolvimento do modelo

### 4.1 Cross Validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
classifiers = {
    "LogisticRegression": LogisticRegression(), # Paramétrico
    "KNN": KNeighborsClassifier(), # Não paramétrico
    "SVC": SVC(), # Não paramétrico
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=17) # Não paramétrico
}

In [ ]:
check_classifiers_scores(classifiers, X_train, y_train)

### 4.2 Avaliando predições do modelo
Escolhendo o modelo com melhores métricas e analisando as predições

In [ ]:
# Escolhendo o modelo com melhor acurácia
lr = classifiers['LogisticRegression']

In [ ]:
#rf.predict(X_train)

predicted_y_proba = lr.predict_proba(X_test)[:, 1]
predicted_y = lr.predict(X_test)

lr_mc = confusion_matrix(y_test, predicted_y)

sns.heatmap(lr_mc, annot=True, cmap=sns.light_palette("black"))
plt.savefig('confusion_matriz_lr.png')

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
plot_roc_curve(y_test, predicted_y_proba, 'roc_curve_lr.png')

### 4.3 Over-sampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
y_train.value_counts()

In [ ]:
oversample = SMOTE(sampling_strategy='minority', random_state=17)
X_res, y_res = oversample.fit_resample(X_train, y_train)

In [ ]:
y_res.value_counts()

In [ ]:
classifiers_v2 ={
    "LogisticRegression": LogisticRegression(), # Paramétrico
    "KNN": KNeighborsClassifier(), # Não paramétrico
    "SVC": SVC(), # Não paramétrico
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=17) # Não paramétrico
}

check_classifiers_scores(classifiers_v2, X_res, y_res)

#### 4.3.1 Anáise modelo com oversample 

In [ ]:
rf = classifiers_v2['RandomForest']

predicted_y_proba = rf.predict_proba(X_test)[:, 1]
predicted_y = rf.predict(X_test)

rf_mc = confusion_matrix(y_test, predicted_y)

sns.heatmap(rf_mc, annot=True, cmap=sns.light_palette("black"))
plt.savefig('confusion_matriz_rf.png')

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
plot_roc_curve(y_test, predicted_y_proba, 'roc_curve_rf.png')

#### 4.3.2 Feature Importance

In [ ]:
show_feature_importance(rf.feature_importances_, featured_df)

In [ ]:
plot_feature_importance(rf.feature_importances_, featured_df, 'feature_importance_rf')

### 4.4 Under-sampling 

In [ ]:
y_train.value_counts()

In [ ]:
undersampling_df = X_train.copy()
undersampling_df['LABEL'] = y_train

In [ ]:
suicide_tries_df = undersampling_df.loc[undersampling_df['LABEL'] == 1]
non_suicide_tries_df = undersampling_df.loc[undersampling_df['LABEL'] == 0][:suicide_tries_df.shape[0]]

normal_distributed_df = pd.concat([suicide_tries_df, non_suicide_tries_df]).sample(frac=1, random_state=42)

normal_distributed_df['LABEL'].value_counts()

In [ ]:
y_under_resambled = normal_distributed_df.pop('LABEL')
X_under_resambled = normal_distributed_df

In [ ]:
classifiers_v3 = {
    "LogisticRegression": LogisticRegression(), # Paramétrico
    "KNN": KNeighborsClassifier(), # Não paramétrico
    "SVC": SVC(), # Não paramétrico
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=17) # Não paramétrico
}

check_classifiers_scores(classifiers_v3, X_under_resambled, y_under_resambled)

#### 4.4.1 Análise modelo com undersampling

In [ ]:
rfus = classifiers_v3['RandomForest']

predicted_y_proba = rfus.predict_proba(X_test)[:, 1]
predicted_y = rfus.predict(X_test)

rfus_mc = confusion_matrix(y_test, predicted_y)

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
plot_roc_curve(y_test, predicted_y_proba, 'roc_curve_rf_undersample.png')

In [ ]:
plot_resampling_comparing(rf_mc, rfus_mc, 'confusion_matriz_comparing.png')

#### 4.4.2 Feature Importance

In [ ]:
show_feature_importance(rfus.feature_importances_, featured_df)

In [ ]:
plot_feature_importance(rfus.feature_importances_, featured_df, 'feature_importance_rfus')

### 4.5 Outras Abordagens
Sessão para tentativas de melhoria do modelo preditivo

#### 4.5.1 Removendo variável de alta relevancia

In [ ]:
X_under_resambled_v2 = X_under_resambled.copy().drop(columns=['Q9_PROCESSADO', 'Q1'])
y_under_resambled_v2 = y_under_resambled.copy()

X_test_v2 = X_test.copy().drop(columns=['Q9_PROCESSADO', 'Q1'])

In [ ]:
classifiers_v4 = {
    "LogisticRegression": LogisticRegression(), # Paramétrico
    "KNN": KNeighborsClassifier(), # Não paramétrico
    "SVC": SVC(probability=True), # Não paramétrico
    "RandomForest": RandomForestClassifier(n_jobs=-1, random_state=17) # Não paramétrico
}

check_classifiers_scores(classifiers_v4, X_under_resambled_v2, y_under_resambled_v2)

In [ ]:
svc = classifiers_v4['SVC']

predicted_y_proba = svc.predict_proba(X_test_v2)[:, 1]
predicted_y = svc.predict(X_test_v2)

svc_mc = confusion_matrix(y_test, predicted_y)

sns.heatmap(svc_mc, annot=True, cmap=sns.light_palette("black"))
plt.savefig('confusion_matriz_svc.png')

In [ ]:
print(classification_report(y_test, predicted_y))

In [ ]:
plot_roc_curve(y_test, predicted_y_proba, 'roc_curve_svc.png')

#### 4.5.2 Tunning de Parâmetros

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
lr_params = {"penalty": ['l1', 'l2'], 'C': [0.01, 0.1, 1, 10, 100]}
knn_params = {"n_neighbors": list(range(2,5,1)), 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}
svm_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}
rf_params = {
    'max_depth': [1, 2, 5, 7, 10, 20],
    'n_estimators': [1, 4, 8, 16, 32, 64, 100, 200],
    'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
    'min_samples_leaf': np.linspace(0.1, 0.5, 5, endpoint=True),
    'criterion': ['entropy', 'gini']
}

classifiers_v5 = {
    "LogisticRegression": GridSearchCV(LogisticRegression(), lr_params, n_jobs=-1), 
    "KNN": GridSearchCV(KNeighborsClassifier(), knn_params, n_jobs=-1),
    "SVC": GridSearchCV(SVC(probability=True), svm_params, n_jobs=-1), 
    "RandomForest": GridSearchCV(RandomForestClassifier(n_jobs=-1, random_state=17), rf_params, n_jobs=-1)
}

In [ ]:
for key, classifier in classifiers_v5.items():
    classifier.fit(X_under_resambled_v2, y_under_resambled_v2)
    best_estimator = classifier.best_estimator_
    
    training_score = cross_val_score(best_estimator, X_under_resambled_v2, y_under_resambled_v2, cv=5, n_jobs=-1)
    precision_score = cross_val_score(best_estimator, X_under_resambled_v2, y_under_resambled_v2, cv=5, n_jobs=-1, scoring='precision')
    recall_score = cross_val_score(best_estimator, X_under_resambled_v2, y_under_resambled_v2, cv=5, n_jobs=-1, scoring='recall')

    print(best_estimator.__class__.__name__, 
          "[Accuracy]:", round(training_score.mean(), 2) * 100, "%", 
          "[Precision]:", round(precision_score.mean(), 2),
          "[Recall]:", round(recall_score.mean(), 2)
    )